In [12]:
import pandas as pd
import numpy as np
from scipy.integrate import simpson as simps
from scipy.signal import find_peaks
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import warnings

In [31]:
# === Load Data ===
boning = pd.read_csv('Boning.csv')
slicing = pd.read_csv('Slicing.csv')

# === Step 1: Filter Required Columns ===
cols = ['Frame', 'Neck x', 'Neck y', 'Neck z', 'Head x', 'Head y', 'Head z']
boning = boning[cols].copy()
slicing = slicing[cols].copy()
boning['class'] = 0
slicing['class'] = 1
df = pd.concat([boning, slicing], ignore_index=True)

# === Step 2: Composite Feature Functions ===
def compute_features(df, prefix):
    x, y, z = df[f'{prefix} x'], df[f'{prefix} y'], df[f'{prefix} z']
    return pd.DataFrame({
        f'{prefix}_rms_xy': np.sqrt(x**2 + y**2),
        f'{prefix}_rms_yz': np.sqrt(y**2 + z**2),
        f'{prefix}_rms_zx': np.sqrt(z**2 + x**2),
        f'{prefix}_rms_xyz': np.sqrt(x**2 + y**2 + z**2),
        f'{prefix}_roll': np.degrees(np.arctan2(y, np.sqrt(x**2 + z**2))),
        f'{prefix}_pitch': np.degrees(np.arctan2(x, np.sqrt(y**2 + z**2)))
    })

neck_features = compute_features(df, 'Neck')
head_features = compute_features(df, 'Head')
df = pd.concat([df, neck_features, head_features], axis=1)


In [17]:
# === Step 3: Feature Computation per Minute (60 Frames) ===
def extract_features(block):
    feats = {}
    for col in block.columns:
        values = block[col].values
        feats[f'{col}_mean'] = np.mean(values)
        feats[f'{col}_std'] = np.std(values)
        feats[f'{col}_min'] = np.min(values)
        feats[f'{col}_max'] = np.max(values)
        feats[f'{col}_auc'] = simps(values)
        feats[f'{col}_peaks'] = len(find_peaks(values)[0])
    return feats

# Exclude frame and class
feature_cols = df.columns.difference(['Frame', 'class'])
samples = []

for start in range(0, len(df), 60):
    end = start + 60
    if end > len(df):
        break
    block = df.iloc[start:end]
    features = extract_features(block[feature_cols])
    features['class'] = block['class'].iloc[0]
    samples.append(features)

features_df = pd.DataFrame(samples)
features_df.reset_index(drop=True, inplace=True)

# === Save final features ===
features_df.to_csv("final_feature_dataset.csv", index=False)
print("Final dataset with 108 features + class label saved as 'final_feature_dataset.csv'")

Final dataset with 108 features + class label saved as 'final_feature_dataset.csv'


In [21]:
# === Step 4: Model Training and Evaluation ===
X = features_df.drop(columns=["class"])
y = features_df["class"]

def evaluate_model(clf, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf.fit(X_train, y_train)
    test_acc = accuracy_score(y_test, clf.predict(X_test))
    cross_val_acc = cross_val_score(clf, X, y, cv=10).mean()
    return round(test_acc * 100, 2), round(cross_val_acc * 100, 2)

svm_results = {}



In [22]:
# 1. Base SVM
svm_results["Original features"] = evaluate_model(SVC(), X, y)

In [23]:
# 2. SVM with Hyperparameter Tuning
param_grid = {'C': [0.1, 1, 10], 'gamma': ['scale', 0.1, 1], 'kernel': ['rbf']}
grid_svm = GridSearchCV(SVC(), param_grid, cv=3)
grid_svm.fit(X, y)
best_svm = grid_svm.best_estimator_
svm_results["With hyper-parameter tuning"] = evaluate_model(best_svm, X, y)

In [24]:
# 3. SVM + Feature Selection
X_selected = SelectKBest(score_func=f_classif, k=10).fit_transform(X, y)
svm_results["With feature selection and hyperparameter tuning"] = evaluate_model(best_svm, X_selected, y)

In [25]:
# 4. SVM + PCA
X_pca = PCA(n_components=10).fit_transform(X)
svm_results["With PCA and hyperparameter tuning"] = evaluate_model(best_svm, X_pca, y)

In [26]:
# === Step 5: Other Classifiers ===
other_results = {
    "SVM": svm_results["With hyper-parameter tuning"],
    "SGD": evaluate_model(SGDClassifier(), X, y),
    "RandomForest": evaluate_model(RandomForestClassifier(), X, y),
    "MLP": evaluate_model(MLPClassifier(max_iter=500), X, y)
}

In [27]:
# === Summary Tables ===
svm_summary = pd.DataFrame(svm_results, index=["Train-test split", "Cross-validation"]).T
other_summary = pd.DataFrame(other_results, index=["Train-test split", "Cross-validation"]).T

print("=== SVM Summary Table ===")
display(svm_summary)

print("\n=== Other Classifiers Summary Table ===")
display(other_summary)

=== SVM Summary Table ===


,Train-test split,Cross-validation
Original features,77.56,75.35
With hyper-parameter tuning,77.56,75.35
With feature selection and hyperparameter tuning,82.55,84.01
With PCA and hyperparameter tuning,77.56,75.35



=== Other Classifiers Summary Table ===


,Train-test split,Cross-validation
SVM,77.56,75.35
SGD,73.68,72.19
RandomForest,85.60,86.10
MLP,73.68,76.52
